<a href="https://colab.research.google.com/github/MOAzeemKhan/MM_Reaction_Analysis/blob/main/MM_Emotional_Detection_Updated_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Approach 1
## About Model: ViT-Face-Expression model from Hugging Face, a transformer-based pre-trained model explicitly designed for emotion detection tasks.

## Using: Facenet_pytorch's MTCNN for accurate face detection and transformers for leveraging the cutting-edge ViT-Face-Expression model.

In [1]:
!pip install facenet-pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.

In [ ]:
!pip install matplotlib

In [ ]:
!pip list

In [ ]:
!pip install seaborn

In [ ]:
!pip install moviepy

In [ ]:
!pip install transformers

In [1]:
path = "/content/drive/MyDrive/MM/Daivik Jayan M video reaction.mov"
# Directory to save frame chunks
#output_dir = "/data/mpstme-azeem/video_frames/"
output_dir = "/content/video_frames/"
# Directory where chunks are saved
#clip_output_dir = "/data/mpstme-azeem/video_clips/"
clip_output_dir = "/content/video_clips/"
#path_cache = "/data/mpstme-azeem"
path_cache = "/content/"

In [2]:
import torch

# Set device to GPU if available, otherwise use CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import seaborn as sns
from tqdm.notebook import tqdm

from moviepy.editor import VideoFileClip, ImageSequenceClip

import torch
from facenet_pytorch import (MTCNN)

from transformers import (AutoFeatureExtractor,
                          AutoModelForImageClassification,
                          AutoConfig)

from PIL import Image, ImageDraw
from moviepy.editor import VideoFileClip
import numpy as np
from tqdm import tqdm
import os

Running on device: cuda:0


In [3]:
def detect_emotions(image):
    """
    Detect emotions from a given image, displays the detected
    face and the emotion probabilities in a bar plot.

    Parameters:
    image (PIL.Image): The input image.

    Returns:
    PIL.Image: The cropped face from the input image.
    """

    # Create a copy of the image to draw on
    temporary = image.copy()
    #plt.imshow(temporary)
    # Use the MTCNN model to detect faces in the image
    sample = mtcnn.detect(temporary)
    print(sample)
    # If a face is detected
    if sample[0] is not None:
        #print("IN IFFFF")
        # Get the bounding box coordinates of the face
        box = sample[0][0]

        # Crop the detected face from the image
        face = temporary.crop(box)

        # Pre-process the cropped face to be fed into the
        # emotion detection model
        inputs = extractor(images=face, return_tensors="pt")

        # Pass the pre-processed face through the model to
        # get emotion predictions
        outputs = model(**inputs)

        # Apply softmax to the logits to get probabilities
        probabilities = torch.nn.functional.softmax(outputs.logits,
                                                    dim=-1)

        # Retrieve the id2label attribute from the configuration
        id2label = AutoConfig.from_pretrained(
            "trpakov/vit-face-expression", cache_dir = path_cache
        ).id2label

        # Convert probabilities tensor to a Python list
        probabilities = probabilities.detach().numpy().tolist()[0]

        # Map class labels to their probabilities
        class_probabilities = {id2label[i]: prob for i,
                               prob in enumerate(probabilities)}

        # Define colors for each emotion
        colors = {
            "angry": "red",
            "disgust": "green",
            "fear": "gray",
            "happy": "yellow",
            "neutral": "purple",
            "sad": "blue",
            "surprise": "orange"
        }
        palette = [colors[label] for label in class_probabilities.keys()]
        #print("IN IFFF")
        # Prepare a figure with 2 subplots: one for the face image,
        # one for the bar plot
        fig, axs = plt.subplots(1, 2, figsize=(15, 6))

        # Display the cropped face in the left subplot
        axs[0].imshow(np.array(face))
        axs[0].axis('off')

        # Create a horizontal bar plot of the emotion probabilities in
        # the right subplot
        sns.barplot(ax=axs[1],
                    y=list(class_probabilities.keys()),
                    x=[prob * 100 for prob in class_probabilities.values()],
                    palette=palette,
                    orient='h')
        axs[1].set_xlabel('Probability (%)')
        axs[1].set_title('Emotion Probabilities')
        axs[1].set_xlim([0, 100])  # Set x-axis limits to show percentages

        # Show the plot
        plt.show()
        return face, class_probabilities
    return None, None

In [4]:
# Initialize MTCNN model for single face cropping
mtcnn = MTCNN(
    image_size=160,
    margin=0,
    min_face_size=200,
    thresholds=[0.6, 0.7, 0.7],
    factor=0.709,
    post_process=True,
    keep_all=False,
    device=device
)
# Load the pre-trained model and feature extractor
extractor = AutoFeatureExtractor.from_pretrained(
    "trpakov/vit-face-expression", cache_dir = path_cache
)
model = AutoModelForImageClassification.from_pretrained(
    "trpakov/vit-face-expression", cache_dir = path_cache
)

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(

  warnings.warn(



In [5]:
def create_combined_image(face, class_probabilities):
    """
    Create an image combining the detected face and a barplot
    of the emotion probabilities.

    Parameters:
    face (PIL.Image): The detected face.
    class_probabilities (dict): The probabilities of each
        emotion class.

    Returns:
    np.array: The combined image as a numpy array.
    """
    # Define colors for each emotion
    colors = {
        "angry": "red",
        "disgust": "green",
        "fear": "gray",
        "happy": "yellow",
        "neutral": "purple",
        "sad": "blue",
        "surprise": "orange"
    }
    palette = [colors[label] for label in class_probabilities.keys()]

    # Create a figure with 2 subplots: one for the
    # face image, one for the barplot
    fig, axs = plt.subplots(1, 2, figsize=(15, 6))

    # Display face on the left subplot
    axs[0].imshow(np.array(face))
    axs[0].axis('off')

    # Create a barplot of the emotion probabilities
    # on the right subplot
    sns.barplot(ax=axs[1],
                y=list(class_probabilities.keys()),
                x=[prob * 100 for prob in class_probabilities.values()],
                palette=palette,
                orient='h')
    axs[1].set_xlabel('Probability (%)')
    axs[1].set_title('Emotion Probabilities')
    axs[1].set_xlim([0, 100])  # Set x-axis limits

    # Convert the figure to a numpy array
    canvas = FigureCanvas(fig)
    canvas.draw()
    img = np.frombuffer(canvas.tostring_rgb(), dtype='uint8')
    img  = img.reshape(canvas.get_width_height()[::-1] + (3,))

    plt.close(fig)
    return img

In [37]:
# Set cache directories for XDG and Hugging Face Hub
os.environ['XDG_CACHE_HOME'] = '/data/mpstme-azeem/msds2023/jlegara/.cache'
os.environ['HUGGINGFACE_HUB_CACHE'] = '/data/mpstme-azeem/msds2023/jlegara/.cache'

In [6]:
# Set cache directories for XDG and Hugging Face Hub
os.environ['XDG_CACHE_HOME'] = '/home/msds2023/jlegara/.cache'
os.environ['HUGGINGFACE_HUB_CACHE'] = '/home/msds2023/jlegara/.cache'

In [39]:
!ls

'daivik_clipped_reaction - Made with Clipchamp.mp4'   sample_data
 drive						      video_chunks
 final_plot.png					      video_frames
 models--trpakov--vit-face-expression


In [7]:
# Load your video
scene = path
clip = VideoFileClip(scene)

# Save video frames per second
vid_fps = clip.fps

In [ ]:
# Get the video (as frames)
video = clip.without_audio()
os.makedirs(output_dir, exist_ok=True)

# Define the batch size and chunk size
batch_size = 32  # Number of frames to process at once
chunk_size = 500  # Number of frames per file

# Temporary list to store a batch of frames
batch_frames = []
chunk_count = 0  # To keep track of the chunk number

# Process video frames
for frame in tqdm(video.iter_frames()):
    batch_frames.append(np.array(frame))

    # When the batch size is reached, store it in the current chunk
    if len(batch_frames) >= batch_size:
        if len(batch_frames) > 0:
            # Save the current batch to the chunk
            if 'current_chunk' in locals():
                current_chunk.extend(batch_frames)
            else:
                current_chunk = batch_frames
        batch_frames = []  # Clear the temporary batch list to save memory

    # If the chunk size is reached, save the chunk to disk and reset
    if 'current_chunk' in locals() and len(current_chunk) >= chunk_size:
        # Save current chunk as a numpy file
        np.save(os.path.join(output_dir, f'video_chunk_{chunk_count}.npy'), np.array(current_chunk))
        chunk_count += 1
        del current_chunk  # Clear the current chunk from memory

# Save any remaining frames in the last chunk
if 'current_chunk' in locals() and len(current_chunk) > 0:
    np.save(os.path.join(output_dir, f'video_chunk_{chunk_count}.npy'), np.array(current_chunk))

# Check how many chunks were created
print(f"Total chunks saved: {chunk_count + 1}")

1021it [05:20, 34.25it/s]

In [ ]:
# Example of loading a chunk back into memory
loaded_chunk = np.load('/data/mpstme-azeem/video_frames/video_chunk_0.npy')
print(f"Loaded chunk shape: {loaded_chunk.shape}")

In [ ]:
# Example of loading a chunk back into memory
loaded_chunk = np.load('/content/video_frames/video_chunk_0.npy')
print(f"Loaded chunk shape: {loaded_chunk.shape}")

In [ ]:
len(loaded_chunk)

In [ ]:
# Choose a frame
frame = loaded_chunk[10]  # choosing the 10th frame

# Convert the frame to a PIL image and display it
image = Image.fromarray(frame)
detect_emotions(image)

In [ ]:
def rest_code(path_chunk, chunk_output_dir, vid_fps):
  # Ensure the output directory exists
  os.makedirs(chunk_output_dir, exist_ok=True)
  skips = 2
  reduced_video = []

  loaded_chunk = np.load(path_chunk)
  print(f"Loaded chunk shape: {loaded_chunk.shape}")
  # Calculate time per frame in the original video (before skips)
  time_per_frame = 1 / vid_fps

  for i in tqdm(range(0, len(loaded_chunk), skips)):
      reduced_video.append(loaded_chunk[i])

  # Define a list of emotions
  emotions = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]

  # List to hold the combined images
  combined_images = []

  # Create a list to hold the class probabilities for all frames
  all_class_probabilities = []
  timestamps = []

  # Loop over video frames
  for i, frame in tqdm(enumerate(reduced_video),
                      total=len(reduced_video),
                      desc="Processing frames"):
      # Convert frame to uint8
      frame = frame.astype(np.uint8)
      #print(frame)
      # Calculate the accurate timestamp for this frame
      # Since we're skipping frames, the timestamp is calculated based on the original frame index (i * skips)
      timestamp = (i * skips) * time_per_frame
      timestamps.append(timestamp)

      # Call detect_emotions to get face and class probabilities
      #print(Image.fromarray(frame))
      face, class_probabilities = detect_emotions(Image.fromarray(frame))

      # If a face was found
      if face is not None:
          # Create combined image for this frame
          combined_image = create_combined_image(face, class_probabilities)

          # Append combined image to the list
          combined_images.append(combined_image)
      else:
          # If no face was found, set class probabilities to None
          class_probabilities = {emotion: None for emotion in emotions}

      # Append class probabilities to the list
      all_class_probabilities.append(class_probabilities)

  # Convert list of images to video clip
  clip_with_plot = ImageSequenceClip(combined_images,
                                    fps=vid_fps/skips)  # Choose the frame rate (fps) according to your requirement

  # Generate unique filenames for this chunk
  chunk_name = os.path.basename(path_chunk).split('.')[0]
  video_output_path = os.path.join(chunk_output_dir, f"{chunk_name}_plot_video.mp4")
  #prob_output_path = os.path.join(chunk_output_dir, f"{chunk_name}_class_probabilities.pkl")
  df_output_path = os.path.join(chunk_output_dir, f"{chunk_name}_class_probabilities.csv")
  #reduced_video_output_path = os.path.join(chunk_output_dir, f"{chunk_name}_reduced_video.npy")

  # Write the video to a file with a specific frame rate
  clip_with_plot.write_videofile(video_output_path, fps=vid_fps/skips)

  # Display the clip
  #clip_with_plot.ipython_display(width=900)
  # Save class probabilities as a pickle file
  #with open(prob_output_path, 'wb') as f:
      #pickle.dump(all_class_probabilities, f)

  # Define colors for each emotion
  colors = {
      "angry": "red",
      "disgust": "green",
      "fear": "gray",
      "happy": "yellow",
      "neutral": "purple",
      "sad": "blue",
      "surprise": "orange"
  }

  # Convert list of class probabilities into a DataFrame
  df = pd.DataFrame(all_class_probabilities)
  df['timestamp'] = timestamps

  # Convert probabilities to percentages
  df = df * 100
  # Save DataFrame to CSV file
  df.to_csv(df_output_path, index=False)

  # Create a line plot
  #plt.figure(figsize=(15,8))
  #for emotion in df.columns:
      #plt.plot(df[emotion], label=emotion, color=colors[emotion])

  #plt.xlabel('Frame Order')
  #plt.ylabel('Emotion Probability (%)')
  #plt.title('Emotion Probabilities Over Time')
  #plt.legend()
  #plt.show()
  # Save plot to file
  #plt.savefig(plot_output_path)
  #plt.close()

  print(f"Processed chunk saved: {video_output_path},{df_output_path}")

In [ ]:
chunk_dir = "/content/video_frames/"
chunk_output_dir = os.path.join("/content/", 'video_chunks/')
npy_files = [f for f in os.listdir(chunk_dir) if f.endswith('.npy')]
npy_files
for file in npy_files:
  rest_code(os.path.join(chunk_dir, file), chunk_output_dir, vid_fps)

In [ ]:
def create_final_dataframe(chunk_output_dir):
    # List all CSV files in the chunk output directory
    csv_files = [f for f in os.listdir(chunk_output_dir) if f.endswith('_class_probabilities.csv')]

    # Initialize an empty DataFrame to store the final merged DataFrame
    final_df = pd.DataFrame()

    # Variable to track the current frame count across all chunks
    frame_count = 0

    # Iterate over each CSV file and concatenate them
    for csv_file in sorted(csv_files):
        # Load the chunk DataFrame
        chunk_df = pd.read_csv(os.path.join(chunk_output_dir, csv_file))

        # Update the index to reflect the correct frame order
        chunk_df.index = range(frame_count, frame_count + len(chunk_df))

        # Append the chunk DataFrame to the final DataFrame
        final_df = pd.concat([final_df, chunk_df], axis=0)

        # Update the frame count
        frame_count += len(chunk_df)

    return final_df

In [ ]:
final_df = create_final_dataframe(chunk_output_dir)
# Define colors for each emotion
colors = {"angry": "red","disgust": "green","fear": "gray","happy": "yellow","neutral": "purple","sad": "blue","surprise": "orange"}
plt.figure(figsize=(15,8))
for emotion in final_df.columns:
    plt.plot(final_df[emotion], label=emotion, color=colors[emotion])

plt.xlabel('Frame Order')
plt.ylabel('Emotion Probability (%)')
plt.title('Emotion Probabilities Over Time')
plt.legend()
plt.show()

plt.savefig("/content/final_plot.png")
plt.close()


final_df_output_path = os.path.join(chunk_output_dir, "final_class_probabilities.csv")
final_df.to_csv(final_df_output_path, index=True)
print(f"Final DataFrame saved to: {final_df_output_path}")